# Convex Hull

In [2]:
import ee
import numpy as np
from IPython.display import Image
import matplotlib.pyplot as plt

# Trigger the authentication flow.
ee.Authenticate()

Enter verification code: 4/2wGdm-HP1nDFT91ACYhIjp99wqz9aTknNndaPbwfjOSqDhREimwG2aY

Successfully saved authorization token.


In [3]:
ee.Initialize()

In [4]:
def get_images(path_list , row_list , satelite , start_date , end_date , max_cloud_percentage):
    
    # get image collection object
    coll = ee.ImageCollection(satelite)\
        .filterDate(start_date, end_date)\
        .filter(ee.Filter.inList('WRS_PATH', path_list))\
        .filter(ee.Filter.inList('WRS_ROW', row_list))\
        .filter(ee.Filter.lt('CLOUD_COVER' , max_cloud_percentage))  # note ~ not less than or equal to

    # get image_id's
    image_ids = list( map( lambda x : x['id'] , coll.getInfo()['features'] ) ) 
    
    # get image objects
    images = list( map( lambda x: ee.Image(x) , image_ids ) )
    
    return images


In [28]:
p = [14]
r = [32]
sat = 'LANDSAT/LC08/C01/T1_TOA'
sd = '2013-05-01'
ed = '2020-05-01'
cc= 5
image_list = get_images(p, r, sat, sd, ed, cc)

In [29]:
lake_bounding_box = ([-74.86 , 40.63 ],\
 [-74.85 , 40.59],\
 [-74.77 , 40.59],\
 [-74.80 , 40.63])


lake_region = ee.Geometry.Rectangle([-74.86 , 40.59 , -74.79 , 40.65])

In [30]:
# this function clip to the lake
def clip(image):
    lake_region = ee.Geometry.Rectangle([-74.86 , 40.59 , -74.79 , 40.65])
    return image.clip(lake_region)

In [31]:
# map the function to our image_list
image_list = list(map(clip, image_list))

In [32]:
# add ndwi and ndvi bands. For some reason, my NDWI is not working properly. (are they correct bands?)
def addNDWI(image):
    return image.addBands(image.normalizedDifference(['B5', 'B7']))


def addNDVI(image):
    return image.addBands(image.normalizedDifference(['B3', 'B5']))

In [33]:
image_list = list(map(addNDVI, image_list))        # set as 'nd'

In [34]:

parameters = {'min': -1.0, # value which is mapped to 0
              'max': 1.0,  # value which is mapped to 255
              'dimensions': 768,   # size of the image 
              'bands': ['nd'],     # The bands we select
              'palette': ['green', 'white', 'blue'],  # only use a pallete when visualizing one.
              #'region': lake_region
}

lake_image = image_list[0]
display(Image(url = lake_image.getThumbUrl(parameters)))

## We only want the pixels which have water.
I found this https://gis.stackexchange.com/questions/298157/earth-engine-filtering-to-non-zero-pixels-surrounded-by-only-zero-value-pixels <br>
var mask = i.neq(0) <br>
var masked = i.updateMask(mask).neq(0)

In [35]:
waterthreshold = .1;
image_ndvi = lake_image.select(['nd'])

watermask = image_ndvi.gte(waterthreshold)
fin = image_ndvi.updateMask(watermask)


### Create multi point of only water pixels (lon and lat coords)

In [36]:
proj = fin.projection()
coord_space = fin.pixelCoordinates(proj)
#Creates a two band image containing the x and y coordinates of each pixel in the given projection.
latlon = fin.pixelLonLat()

latlon = latlon.updateMask(watermask)


coords = latlon.select(['longitude', 'latitude'])\
             .reduceRegion(\
 reducer = ee.Reducer.toList(),\
 geometry = lake_region,\
 scale = 30)


lat = ee.List(coords.get('latitude'))
lon = ee.List(coords.get('longitude'))

# zip them. Example: zip([1, 3],[2, 4]) --> [[1, 2], [3,4]]
point_list = lon.zip(lat)
mp = ee.Geometry.MultiPoint(point_list)


In [37]:
latlon.bandNames().getInfo()

['longitude', 'latitude']

In [38]:
mp.getInfo()

{'type': 'MultiPoint',
 'coordinates': [[-74.81398756544826, 40.6003024968301],
  [-74.81371807086303, 40.6003024968301],
  [-74.81344857627779, 40.6003024968301],
  [-74.81614352213015, 40.600571991415336],
  [-74.81587402754491, 40.600571991415336],
  [-74.81560453295968, 40.600571991415336],
  [-74.81533503837444, 40.600571991415336],
  [-74.8150655437892, 40.600571991415336],
  [-74.81479604920396, 40.600571991415336],
  [-74.81452655461874, 40.600571991415336],
  [-74.8142570600335, 40.600571991415336],
  [-74.81398756544826, 40.600571991415336],
  [-74.81371807086303, 40.600571991415336],
  [-74.81344857627779, 40.600571991415336],
  [-74.81317908169255, 40.600571991415336],
  [-74.81290958710731, 40.600571991415336],
  [-74.81264009252209, 40.600571991415336],
  [-74.81695200588585, 40.600841486000576],
  [-74.81668251130063, 40.600841486000576],
  [-74.81641301671539, 40.600841486000576],
  [-74.81614352213015, 40.600841486000576],
  [-74.81587402754491, 40.600841486000576],
  

### Get convex Hull

In [39]:
convex_hull = mp.convexHull()

In [40]:
type(convex_hull)

ee.geometry.Geometry

In [86]:
# This Returns the perimeter of the convex hull in meters
convex_hull.perimeter().getInfo()

12768.310044938327

In [42]:
# I would Like to make my own class for this.
import folium 
class Map_Display():

    def __init__(self , location , zoom , height  ):
      self.m =  folium.Map(location= location , zoom_start = zoom)


    def add_ee_layer(self, ee_image_object, vis_params, name):
        map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
        folium.raster_layers.TileLayer(
        tiles = map_id_dict['tile_fetcher'].url_format,
        attr = "Map Data © Google Earth Engine",
        name = name,
        overlay = True,
        control = True
        ).add_to(self.m)

    def add_child(self , child_object):
      self.m.add_child(child_object)

    def show_map(self):
      display(self.m)
    
    

In [43]:

# Test it out
md = Map_Display([40.6, -74.85] , zoom = 12 , height = 500)
#md = Map_Display([4149 , 2524] , zoom = 12 , height = 500)


# Set visualization parameters.
vis_params_W = {
  'min': 0.5,
  'max': 1.0,
  #'palette': ['00FFFF', '0000FF']
}

vis_params_V = {
  'min': -1,
  'max': 1,
  #'palette': ['#d73027', '#f46d43', '#fdae61','#fee08b', '#d9ef8b', '#a6d96a', '#66bd63', '#1a9850']
  #'palette': ['blue', 'white', 'green']
    #'palette': ['blue' , 'white' , 'green']
  }


In [44]:
md.show_map()

In [45]:
folium.GeoJson(
            data = convex_hull.getInfo(),
            name = 'convex_hull',
            overlay = True,
            control = True,
        ).add_to(md.m)

In [46]:
md.show_map()

## Edge Detection

In [47]:
zc = ee.Algorithms.CannyEdgeDetector(
  image = watermask,
  threshold = 0.5)

In [48]:
parameters = {'min': 0,
              'max': 1,
              'dimensions': 768,
              'palette': ['white', 'grey', 'blue'],
}
Image(url = zc.getThumbUrl(parameters))

In [52]:
new_proj = zc.projection()
new_coord_space = zc.pixelCoordinates(new_proj)
#Creates a two band image containing the x and y coordinates of each pixel in the given projection.
new_latlon = zc.pixelLonLat()

new_latlon = new_latlon.updateMask(watermask)


new_coords = new_latlon.select(['longitude', 'latitude'])\
             .reduceRegion(\
 reducer = ee.Reducer.toList(),\
 geometry = lake_region,\
 scale = 30)


new_lat = ee.List(new_coords.get('latitude'))
new_lon = ee.List(new_coords.get('longitude'))

# zip them. Example: zip([1, 3],[2, 4]) --> [[1, 2], [3,4]]
new_point_list = new_lon.zip(new_lat)
new_mp = ee.Geometry.MultiPoint(new_point_list)


In [111]:
new_mp.getInfo()

{'type': 'MultiPoint',
 'coordinates': [[-74.81398756544826, 40.6003024968301],
  [-74.81371807086303, 40.6003024968301],
  [-74.81344857627779, 40.6003024968301],
  [-74.81614352213015, 40.600571991415336],
  [-74.81587402754491, 40.600571991415336],
  [-74.81560453295968, 40.600571991415336],
  [-74.81533503837444, 40.600571991415336],
  [-74.8150655437892, 40.600571991415336],
  [-74.81479604920396, 40.600571991415336],
  [-74.81452655461874, 40.600571991415336],
  [-74.8142570600335, 40.600571991415336],
  [-74.81398756544826, 40.600571991415336],
  [-74.81371807086303, 40.600571991415336],
  [-74.81344857627779, 40.600571991415336],
  [-74.81317908169255, 40.600571991415336],
  [-74.81290958710731, 40.600571991415336],
  [-74.81264009252209, 40.600571991415336],
  [-74.81695200588585, 40.600841486000576],
  [-74.81668251130063, 40.600841486000576],
  [-74.81641301671539, 40.600841486000576],
  [-74.81614352213015, 40.600841486000576],
  [-74.81587402754491, 40.600841486000576],
  

In [163]:
geometries = mp.geometries()
pt1 = geometries.get(0).getInfo()

print(geometries.length().getInfo())
#
# Adding the point breks it
#geometries = geometries.add(new_pt)

print(geometries.length().getInfo())
lr = ee.Geometry.LinearRing(geometries)
pg = ee.Geometry.Polygon(geometries)

13083
13083


In [164]:
pg.perimeter().getInfo()

621906.4672278521

In [178]:

# Test it out
md2 = Map_Display([40.6, -74.85] , zoom = 12 , height = 500)
#md = Map_Display([4149 , 2524] , zoom = 12 , height = 500)


# Set visualization parameters.
vis_params_W = {
  'min': 0.5,
  'max': 1.0,
  #'palette': ['00FFFF', '0000FF']
}

vis_params_V = {
  'min': -1,
  'max': 1,
  #'palette': ['#d73027', '#f46d43', '#fdae61','#fee08b', '#d9ef8b', '#a6d96a', '#66bd63', '#1a9850']
  #'palette': ['blue', 'white', 'green']
    #'palette': ['blue' , 'white' , 'green']
  }


In [179]:
folium.GeoJson(
            data = pg.getInfo(),
            name = 'pg',
            overlay = True,
            control = True,
        ).add_to(md2.m)

In [180]:
md2.show_map()

### Thoughts:
The order is wrong here. Can I get the center point ~ convert to radial coordinate system. Then sort by the angle value. ~ Then make the polygon and see how it works.